In [1]:
# Instale as bibliotecas necessárias
!pip install --upgrade google-generativeai feedparser matplotlib fpdf


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 73.2 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=2cfc0faff9e313806fb8a2137660e4d8e7b3e166f1b49471d618f84ebc19a983
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=062a0e80fe38cdfc756225f8469917d886a5d6517caf3dc6df4babc026f37e44
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built fpdf sgmllib3k
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [4]:
# Importando as bibliotecas
import feedparser
import matplotlib.pyplot as plt
from fpdf import FPDF
import getpass

# Para Google Gemini (Gemini API)
import google.generativeai as genai

In [5]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.9 MB/s eta 0:00:00


In [7]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [8]:
!pip install -q google-generativeai

In [9]:
# Importando as bibliotecas
import feedparser
import matplotlib.pyplot as plt
from fpdf import FPDF
import getpass
import urllib.parse
import re # Importar a biblioteca re para contar classificações

# Para Google Gemini (Gemini API)
import google.generativeai as genai

import os
from google.colab import userdata

# Carregar a API Key do userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [10]:
# AGENTE 0: Cadastro de Cliente

display(HTML("""
<div style="background: linear-gradient(90deg, #6f42c1, #0d6efd); padding:20px; border-radius:12px; margin-bottom:18px;">
  <h2 style="color:white; text-align:center; margin:0; font-family: Arial, sans-serif; letter-spacing:1px;">
    Cadastro de Cliente - Clipping Inteligente
  </h2>
  <p style="color:#d1e6fa; text-align:center; margin:6px 0 0 0;">Digite as informações do cliente para buscar notícias automaticamente.</p>
</div>
"""))

clientes = []

nome_cliente = widgets.Text(
    value='',
    placeholder='Nome do cliente',
    description='Nome:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='400px')
)

palavras_chave = widgets.Text(
    value='',
    placeholder='Palavras-chave separadas por vírgula',
    description='Keywords:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='400px')
)

botao_cadastrar = widgets.Button(
    description="Cadastrar Cliente",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#6f42c1', 'font_weight':'bold'}
)

saida = widgets.Output()

def on_cadastrar_clicked(b):
    with saida:
        clear_output()
        nome = nome_cliente.value.strip()
        palavras = [k.strip() for k in palavras_chave.value.split(',') if k.strip()]
        if nome and palavras:
            clientes.append({'nome': nome, 'keywords': palavras})
            print(f"\033[94mCliente cadastrado com sucesso!\033[0m")
            print("Clientes cadastrados:")
            for idx, cliente in enumerate(clientes, 1):
                print(f"{idx}. {cliente['nome']} - {', '.join(cliente['keywords'])}")
        else:
            print("\033[91mPreencha todos os campos!\033[0m")

botao_cadastrar.on_click(on_cadastrar_clicked)

display(nome_cliente)
display(palavras_chave)
display(botao_cadastrar)
display(saida)

Text(value='', description='Nome:', layout=Layout(width='400px'), placeholder='Nome do cliente', style=Descrip…

Text(value='', description='Keywords:', layout=Layout(width='400px'), placeholder='Palavras-chave separadas po…

Button(description='Cadastrar Cliente', layout=Layout(width='400px'), style=ButtonStyle(button_color='#6f42c1'…

Output()

In [11]:
# AGENTE 1: Busca de Notícias do Cliente

display(HTML("""
<div style="background: linear-gradient(90deg, #6f42c1, #0d6efd); padding:16px; border-radius:12px; margin-bottom:18px;">
  <h2 style="color:white; text-align:center; margin:0; font-family: Arial, sans-serif; letter-spacing:1px;">
    Seleção de Cliente • Busca de Notícias
  </h2>
  <p style="color:#d1e6fa; text-align:center; margin:6px 0 0 0;">Escolha o cliente e busque as notícias mais recentes dos principais portais.</p>
</div>
"""))

drop_clientes = widgets.Dropdown(
    options=[(f"{c['nome']} - {', '.join(c['keywords'])}", idx) for idx, c in enumerate(clientes)],
    description='Cliente:',
    style={'description_width': '75px'},
    layout=widgets.Layout(width='400px')
)

botao_buscar = widgets.Button(
    description="Buscar Notícias",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#0d6efd', 'font_weight':'bold'}
)

saida_noticias = widgets.Output()
noticias_cliente = {}

def buscar_google_news(cliente, idioma='pt-BR'):
    keywords = ' '.join(cliente['keywords'])
    query = f"{keywords} site:.br"
    query_encoded = urllib.parse.quote(query)
    url = f"https://news.google.com/rss/search?q={query_encoded}&hl={idioma}&gl=BR&ceid=BR:{idioma.split('-')[0].upper()}"
    feed = feedparser.parse(url)
    return feed.entries[:10]

def buscar_g1(cliente):
    url = "https://g1.globo.com/rss/g1/"
    feed = feedparser.parse(url)
    resultados = []
    for entry in feed.entries:
        titulo = entry.title.lower()
        resumo = entry.summary.lower() if hasattr(entry, "summary") else ""
        for kw in cliente['keywords']:
            if kw.lower() in titulo or kw.lower() in resumo:
                resultados.append(entry)
                break
    return resultados[:5]

def buscar_folha(cliente):
    url = "https://feeds.folha.uol.com.br/emcimadahora/rss091.xml"
    feed = feedparser.parse(url)
    resultados = []
    for entry in feed.entries:
        titulo = entry.title.lower()
        resumo = entry.summary.lower() if hasattr(entry, "summary") else ""
        for kw in cliente['keywords']:
            if kw.lower() in titulo or kw.lower() in resumo:
                resultados.append(entry)
                break
    return resultados[:5]

def buscar_noticias_multifonte(cliente):
    noticias = []
    noticias += buscar_google_news(cliente)
    noticias += buscar_g1(cliente)
    noticias += buscar_folha(cliente)
    vistos = set()
    noticias_unicas = []
    for n in noticias:
        if n.title not in vistos:
            noticias_unicas.append(n)
            vistos.add(n.title)
    return noticias_unicas[:15]

def on_buscar_clicked(b):
    with saida_noticias:
        clear_output()
        idx_cliente = drop_clientes.value
        cliente = clientes[idx_cliente]
        noticias = buscar_noticias_multifonte(cliente)
        noticias_cliente[idx_cliente] = noticias
        if noticias:
            display(HTML(f'<div style="color:#fff; background:#0d6efd; padding:8px; border-radius:6px; margin-bottom:12px;">{len(noticias)} notícias encontradas para <b>{cliente["nome"]}</b>!</div>'))
            for idx, noticia in enumerate(noticias):
                display(HTML(f"""
                <div style="background: #f5f6fa; border-left: 4px solid #6f42c1; padding:12px; margin-bottom:10px; border-radius:8px;">
                  <span style="color:#0d6efd; font-weight:bold;">{idx+1}.</span>
                  <span style="color:#222; font-size:1.08em; font-weight:bold;">{noticia.title}</span><br>
                  <a href="{noticia.link}" target="_blank" style="color:#6f42c1; text-decoration:underline;">Acessar notícia</a>
                </div>
                """))
        else:
            display(HTML('<div style="color:#fff; background:#6f42c1; padding:8px; border-radius:6px;">Nenhuma notícia encontrada para as palavras-chave deste cliente.</div>'))

botao_buscar.on_click(on_buscar_clicked)

display(drop_clientes)
display(botao_buscar)
display(saida_noticias)

Dropdown(description='Cliente:', layout=Layout(width='400px'), options=(('Thermas dos Laranjais - Thermas dos …

Button(description='Buscar Notícias', layout=Layout(width='400px'), style=ButtonStyle(button_color='#0d6efd', …

Output()

In [14]:
# AGENTE 2: Resumo das Notícias com IA Gemini

botao_resumir = widgets.Button(
    description="Gerar Resumo com IA",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#6f42c1', 'font_weight':'bold'}
)
saida_resumo = widgets.Output()
resumo = None

def resumir_noticias_gemini(noticias, max_noticias=5):
    textos = "\n".join([
        f"Título: {n.title}\nResumo: {getattr(n, 'summary', '') or getattr(n, 'description', '') or '(Sem resumo disponível)'}"
        for n in noticias[:max_noticias]
    ])
    prompt = (
        "Resuma em tópicos claros, diretos e em linguagem acessível as notícias abaixo, destacando sempre a ligação com o cliente. "
        "Se houver informações repetidas, una em apenas um tópico. Use bullet points (•) para separar os pontos principais.\n\n"
        f"{textos}"
    )
    model = genai.GenerativeModel('gemini-2.0-flash')
    resposta = model.generate_content(prompt)
    return resposta.text

def on_resumir_clicked(b):
    global resumo
    with saida_resumo:
        clear_output()
        idx_cliente = drop_clientes.value
        noticias = noticias_cliente.get(idx_cliente, [])
        if not noticias:
            display(HTML('<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Nenhuma notícia disponível para resumir. Faça a busca antes.</div>'))
            return
        display(HTML('<div style="color:#fff; background:#0d6efd; padding:8px; border-radius:6px; margin-bottom:10px;">Gerando resumo com IA... Aguarde alguns segundos.</div>'))
        try:
            resumo = resumir_noticias_gemini(noticias)
            display(HTML(f"""
            <div style="background: #f5f6fa; border-left: 4px solid #6f42c1; padding:14px; margin-bottom:10px; border-radius:8px;">
              <span style="color:#6f42c1; font-weight:bold; font-size:1.15em;">Resumo Gerado:</span><br>
              <pre style="white-space: pre-wrap; color:#222; font-size:1.05em; background:none; border:none; margin:0;">{resumo}</pre>
            </div>
            """))
        except Exception as e:
            display(HTML(f'<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Erro ao gerar resumo: {str(e)}</div>'))

botao_resumir.on_click(on_resumir_clicked)

display(botao_resumir)
display(saida_resumo)


Button(description='Gerar Resumo com IA', layout=Layout(width='400px'), style=ButtonStyle(button_color='#6f42c…

Output()

In [17]:
# AGENTE 3: Classificação de Sentimento e Dicas

MODELO_GEMINI = "gemini-2.0-flash"


# AGENTE 3: Classificação de Sentimento e Dicas

botao_classificar = widgets.Button(
    description="Classificar Notícias",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#ffc107', 'font_weight':'bold'}
)
saida_classificacao = widgets.Output()
classificacoes_noticias = None

def classificar_noticias(resumo_texto):
    model = genai.GenerativeModel(MODELO_GEMINI)
    prompt = (
        "Classifique cada notícia do resumo abaixo como positiva, neutra ou negativa, explicando o motivo em uma frase. "
        "Retorne a classificação em formato de lista Python contendo dicionários com as chaves 'noticia', 'sentimento' e 'motivo'. "
        "Exemplo:\n"
        "[{'noticia': 'Título...', 'sentimento': 'positiva', 'motivo': '...'}]"
        "Não coloque nenhum bloco de código ou markdown, só a lista Python."
    )
    resposta = model.generate_content(prompt + "\n\n" + resumo_texto)
    texto = resposta.text.strip()
    # Remove blocos de markdown ```python e ``` se existirem
    if texto.startswith("```"):
        texto = texto.split("```")[1]
    texto = texto.strip("` \n")
    try:
        import ast
        texto = texto.replace('•', '-').strip()
        if not texto.startswith('['):
            texto = '[' + texto + ']'
        return ast.literal_eval(texto)
    except Exception as e:
        print(f"Erro ao avaliar a resposta do modelo: {e}")
        print(f"Resposta bruta: {resposta.text}")
        return resposta.text

def classificar_noticias_e_exibir(b):
    global resumo, classificacoes_noticias
    with saida_classificacao:
        clear_output()
        if not resumo:
            display(HTML('<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Nenhum resumo disponível para classificar. Gere o resumo primeiro.</div>'))
            return
        display(HTML('<div style="color:#fff; background:#ffc107; padding:8px; border-radius:6px; margin-bottom:10px;">Classificando as notícias... Aguarde.</div>'))
        try:
            classificacoes_noticias = classificar_noticias(resumo)
            exibicao = ""
            positivas_count = neutras_count = negativas_count = 0
            if isinstance(classificacoes_noticias, list):
                for item in classificacoes_noticias:
                    if isinstance(item, dict) and 'noticia' in item and 'sentimento' in item and 'motivo' in item:
                        exibicao += f"<b>Notícia:</b> {item['noticia']}<br>"
                        exibicao += f"<b>Sentimento:</b> {item['sentimento'].capitalize()}<br>"
                        exibicao += f"<b>Motivo:</b> {item['motivo']}<br><hr>"
                        s = item['sentimento'].lower()
                        if s == 'positiva': positivas_count += 1
                        elif s == 'neutra': neutras_count += 1
                        elif s == 'negativa': negativas_count += 1
                display(HTML(f"""
                <div style="background: #e9ecef; border-left: 4px solid #ffc107; padding:14px; margin-bottom:10px; border-radius:8px;">
                  <span style="color:#ffc107; font-weight:bold; font-size:1.15em;">Classificação das Notícias:</span><br>
                  <pre style="white-space: pre-wrap; color:#333; font-size:1.05em; background:none; border:none; margin:0;">{exibicao}</pre>
                </div>
                """))
                # Gráfico
                plt.figure(figsize=(6, 4))
                labels = ['Positivas', 'Neutras', 'Negativas']
                valores = [positivas_count, neutras_count, negativas_count]
                plt.bar(labels, valores, color=['green', 'gray', 'red'])
                plt.title('Classificação das Notícias', fontsize=14)
                plt.ylabel('Número de Notícias', fontsize=10)
                plt.show()
            else:
                display(HTML(f'<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Erro na classificação. Resposta bruta:<pre>{classificacoes_noticias}</pre></div>'))
        except Exception as e:
            display(HTML(f'<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Erro ao classificar as notícias: {str(e)}</div>'))

botao_classificar.on_click(classificar_noticias_e_exibir)
display(botao_classificar)
display(saida_classificacao)

# Dicas de reputação
botao_dicas = widgets.Button(
    description="Gerar Dicas de Reputação",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#28a745', 'font_weight':'bold'}
)
saida_dicas = widgets.Output()
dicas = None

def dicas_reputacao(classificacoes_lista):
    if not isinstance(classificacoes_lista, list):
        return "Não foi possível gerar dicas: classificação não está no formato esperado."
    model = genai.GenerativeModel(MODELO_GEMINI)
    dicas_gerais = []
    for item in classificacoes_lista:
        if isinstance(item, dict) and 'noticia' in item and 'sentimento' in item and 'motivo' in item:
            prompt_especifico = f"""
            Considerando a seguinte notícia e sua classificação de sentimento, forneça dicas de assessoria de imprensa como um especialista.
            As dicas devem ser concisas e relevantes para a ação a ser tomada, focando em O QUE fazer.
            Use bullet points (•) para listar as dicas.

            Notícia: {item['noticia']}
            Sentimento: {item['sentimento'].capitalize()}
            Motivo: {item['motivo']}

            Dicas de Assessoria de Imprensa:
            """
            try:
                resposta = model.generate_content(prompt_especifico)
                dicas_gerais.append({
                    'noticia': item['noticia'],
                    'sentimento': item['sentimento'],
                    'dicas': resposta.text
                })
            except Exception as e:
                dicas_gerais.append({
                    'noticia': item['noticia'],
                    'sentimento': item['sentimento'],
                    'dicas': f"Erro ao gerar dicas para esta notícia: {str(e)}"
                })
        else:
            dicas_gerais.append({
                'noticia': 'Item com formato inesperado',
                'sentimento': 'indefinido',
                'dicas': f"Não foi possível gerar dicas para o item com formato inesperado: {item}"
            })
    return dicas_gerais

def gerar_dicas_e_exibir(b):
    global classificacoes_noticias, dicas
    with saida_dicas:
        clear_output()
        if not classificacoes_noticias or not isinstance(classificacoes_noticias, list):
            display(HTML('<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Nenhuma classificação válida disponível. Classifique as notícias primeiro.</div>'))
            return
        display(HTML('<div style="color:#fff; background:#28a745; padding:8px; border-radius:6px; margin-bottom:10px;">Gerando dicas de reputação... Aguarde.</div>'))
        try:
            dicas = dicas_reputacao(classificacoes_noticias)
            exibicao_dicas = ""
            if isinstance(dicas, list) and dicas:
                for item in dicas:
                    if isinstance(item, dict) and 'noticia' in item and 'sentimento' in item and 'dicas' in item:
                        exibicao_dicas += f"<b>Notícia:</b> {item['noticia']}<br>"
                        exibicao_dicas += f"<b>Sentimento:</b> {item['sentimento'].capitalize()}<br>"
                        exibicao_dicas += f"<span style='color:#28a745; font-weight:bold;'>Dicas:</span><br><pre style='white-space: pre-wrap; color:#333; font-size:1.05em; background:none; border:none; margin-left:10px;'>{item['dicas']}</pre><hr>"
                display(HTML(f"""
                <div style="background: #e9ecef; border-left: 4px solid #28a745; padding:14px; margin-bottom:10px; border-radius:8px;">
                  <span style="color:#28a745; font-weight:bold; font-size:1.15em;">Dicas de Reputação:</span><br>
                  {exibicao_dicas}
                </div>
                """))
            else:
                display(HTML(f'<div style="color:#fff; background:#6f42c1; padding:8px; border-radius:6px;">{dicas if isinstance(dicas, str) else "Nenhuma dica gerada ou formato inesperado."}</div>'))
        except Exception as e:
            display(HTML(f'<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">Erro ao gerar dicas de reputação: {str(e)}</div>'))

botao_dicas.on_click(gerar_dicas_e_exibir)
display(botao_dicas)
display(saida_dicas)



Button(description='Classificar Notícias', layout=Layout(width='400px'), style=ButtonStyle(button_color='#ffc1…

Output()

Button(description='Gerar Dicas de Reputação', layout=Layout(width='400px'), style=ButtonStyle(button_color='#…

Output()

In [27]:
import unicodedata

def limpa_para_pdf(texto):
    if not texto:
        return ""
    texto = texto.replace("•", "-").replace("*", "-").replace("—", "-").replace("–", "-")
    texto = texto.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")
    texto = ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if ord(c) < 256
    )
    return texto

botao_relatorio = widgets.Button(
    description="Gerar Relatório PDF",
    button_style='',
    layout=widgets.Layout(width='400px'),
    style={'button_color': '#dc3545', 'font_weight':'bold'}
)
saida_relatorio = widgets.Output()

def gerar_relatorio_pdf(b):
    global resumo, classificacoes_noticias, dicas

    with saida_relatorio:
        clear_output()

        if not resumo or not classificacoes_noticias or not isinstance(classificacoes_noticias, list):
            display(HTML(
                '<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">'
                'Resumo e Classificação necessários para gerar o relatório.</div>'))
            return

        if dicas is None or not isinstance(dicas, list):
            try:
                dicas_local = dicas_reputacao(classificacoes_noticias)
                if isinstance(dicas_local, str):
                    dicas_local = "Não foi possível gerar as dicas para o relatório."
            except Exception as e:
                dicas_local = "Erro ao gerar as dicas para o relatório."
        else:
            dicas_local = dicas

        try:
            # Prepara textos
            classificacao_texto_pdf = ""
            for item in classificacoes_noticias:
                if isinstance(item, dict) and 'noticia' in item and 'sentimento' in item and 'motivo' in item:
                    classificacao_texto_pdf += f"- Notícia: {item['noticia']}\n  Sentimento: {item['sentimento'].capitalize()}\n  Motivo: {item['motivo']}\n\n"
                else:
                    classificacao_texto_pdf += f"- Item inesperado: {item}\n\n"

            dicas_texto_pdf = ""
            if isinstance(dicas_local, list):
                for item in dicas_local:
                    if isinstance(item, dict) and 'noticia' in item and 'dicas' in item:
                        dicas_texto_pdf += f"- Notícia: {item['noticia']}\n  Dicas: {item['dicas']}\n\n"
                    else:
                        dicas_texto_pdf += f"- Dica inesperada: {item}\n\n"
            else:
                dicas_texto_pdf = str(dicas_local)

            positivas = sum(1 for item in classificacoes_noticias if isinstance(item, dict) and item.get('sentimento', '').lower() == 'positiva')
            neutras = sum(1 for item in classificacoes_noticias if isinstance(item, dict) and item.get('sentimento', '').lower() == 'neutra')
            negativas = sum(1 for item in classificacoes_noticias if isinstance(item, dict) and item.get('sentimento', '').lower() == 'negativa')

            plt.figure(figsize=(6, 4))
            labels = ['Positivas', 'Neutras', 'Negativas']
            valores = [positivas, neutras, negativas]
            cores = ['#28a745', '#6c757d', '#dc3545']
            plt.bar(labels, valores, color=cores)
            plt.title('Classificação das Notícias', fontsize=15, color='#222')
            plt.ylabel('Número de Notícias', fontsize=12)
            plt.xticks(fontsize=12)
            plt.yticks(range(0, max(valores) + 2 if valores else 2), fontsize=10)
            plt.grid(axis='y', linestyle='--', alpha=0.5)
            plt.tight_layout()
            plt.savefig("grafico.png", bbox_inches='tight')
            plt.close()

            from fpdf import FPDF
            import datetime

            pdf = FPDF()
            pdf.add_page()
            pdf.set_auto_page_break(auto=True, margin=12)
            pdf.set_font("Arial", size=14)

            # Título
            pdf.set_fill_color(111, 66, 193)
            pdf.set_text_color(255, 255, 255)
            pdf.cell(0, 16, txt="Relatório Executivo de Clipping Inteligente", ln=1, align='C', fill=True)
            pdf.ln(3)

            pdf.set_text_color(80, 80, 80)
            pdf.set_font("Arial", size=10)
            pdf.cell(0, 7, txt="Data de geração: ", ln=1)
            pdf.set_font("Arial", style='I', size=10)
            pdf.cell(0, 7, txt=datetime.datetime.now().strftime("%d/%m/%Y %H:%M"), ln=1)
            pdf.ln(2)

            # Resumo
            pdf.set_font("Arial", 'B', size=12)
            pdf.set_text_color(33, 37, 41)
            pdf.cell(0, 8, txt="Resumo das Notícias:", ln=1)
            pdf.set_font("Arial", size=11)
            pdf.multi_cell(0, 6, limpa_para_pdf(resumo))
            pdf.ln(3)

            # Classificação
            pdf.set_font("Arial", 'B', size=12)
            pdf.cell(0, 8, txt="Classificação de Sentimento:", ln=1)
            pdf.set_font("Arial", size=10)
            pdf.multi_cell(0, 6, limpa_para_pdf(classificacao_texto_pdf))
            pdf.ln(3)

            # Dicas
            pdf.set_font("Arial", 'B', size=12)
            pdf.cell(0, 8, txt="Dicas de Comunicação e Alertas:", ln=1)
            pdf.set_font("Arial", size=10)
            pdf.multi_cell(0, 6, limpa_para_pdf(dicas_texto_pdf))
            pdf.ln(3)

            # Gráfico
            pdf.set_font("Arial", 'B', size=12)
            pdf.cell(0, 8, txt="Análise Visual:", ln=1, align='C')
            if os.path.exists("grafico.png"):
                pos_x = (pdf.w - 120) / 2
                pdf.image("grafico.png", x=pos_x, y=pdf.get_y(), w=120)
                pdf.ln(90)
            else:
                pdf.set_text_color(220, 53, 69)
                pdf.cell(0, 10, txt="Erro: Imagem do gráfico não encontrada.", ln=1, align='C')
                pdf.set_text_color(0, 0, 0)
            pdf.ln(2)

            # Rodapé
            pdf.set_y(-22)
            pdf.set_font("Arial", style='I', size=9)
            pdf.set_text_color(150, 150, 150)
            pdf.cell(0, 8, limpa_para_pdf("Relatório gerado automaticamente pelo sistema Clipping Inteligente (Powered by Gemini AI)"), 0, 0, 'C')

            # Salvar PDF
            nome_pdf = "relatorio_clipping.pdf"
            pdf.output(nome_pdf)

            # Baixar PDF automaticamente no Google Colab
            from google.colab import files
            files.download(nome_pdf)

            display(HTML(
                f'<div style="color:#fff; background:#28a745; padding:14px; border-radius:8px; font-size:1.09em;">'
                f'Relatório PDF gerado com sucesso e pronto para download!</div>'))

        except Exception as e:
            display(HTML(
                f'<div style="color:#fff; background:#c5004b; padding:8px; border-radius:6px;">'
                f'Erro ao gerar o relatório PDF: {str(e)}</div>'))

botao_relatorio.on_click(gerar_relatorio_pdf)
display(botao_relatorio)
display(saida_relatorio)





Button(description='Gerar Relatório PDF', layout=Layout(width='400px'), style=ButtonStyle(button_color='#dc354…

Output()